# Bedrock Agents with Return of Control (RoC)

In this notebook, we show how to create a Bedrock Agent with RoC and then use the Agent to invoke the tools defined with LangChain. `langchain-aws` library provides a BedrockAgentsRunnable which can be used with LangChain's AgentExecutor. 

### Prerequisites:
1. Set your aws credentials for your environment, example: https://docs.aws.amazon.com/cli/v1/userguide/cli-configure-envvars.html#envvars-set.
1. Ensure that langchain, langgraph are installed in the environment and that the local langchain-aws is accessible from the path or installed into the environment. 

## Example 1: Create a mortgage agent that determines the interest rate
In this example, we create a mortgage agent with two tools. The first tool will return the asset values of a given asset holder. The second tool will return the interest rate for a given asset holder with a given asset value.

### Step 1: Define the tools for the agent

### Step 2: Define the foundation model and instructions for the agent

In [21]:
foundational_model = 'anthropic.claude-3-sonnet-20240229-v1:0'
foundational_model

'anthropic.claude-3-sonnet-20240229-v1:0'

In [22]:
instructions="You are an agent who helps with getting the mortgage rate based on the current asset valuation"
instructions

'You are an agent who helps with getting the mortgage rate based on the current asset valuation'

### Step 3: Define the resource role to use with the Bedrock Agent

In [23]:
import boto3
import json
import time
import uuid

def _create_agent_role(
        agent_region,
        foundational_model
) -> str:
    """
    Create agent resource role prior to creation of agent, at this point we do not have agentId, keep it as wildcard

    Args:
        agent_region: AWS region in which is the Agent if available
        foundational_model: The model used for inference in AWS BedrockAgents
    Returns:
       Agent execution role arn
    """
    try:
        account_id = boto3.client('sts').get_caller_identity().get('Account')
        assume_role_policy_document = json.dumps({
            "Version": "2012-10-17",
            "Statement": [
                {
                    "Effect": "Allow",
                    "Principal": {
                        "Service": "bedrock.amazonaws.com"
                    },
                    "Action": "sts:AssumeRole",
                    "Condition": {
                        "ArnLike": {
                            "aws:SourceArn": f"arn:aws:bedrock:{agent_region}:{account_id}:agent/*"
                        }
                    }
                }
            ]
        })
        managed_policy = {
            "Version": "2012-10-17",
            "Statement": [
                {
                    "Sid": "AmazonBedrockAgentBedrockFoundationModelStatement",
                    "Effect": "Allow",
                    "Action": "bedrock:InvokeModel",
                    "Resource": [
                        f"arn:aws:bedrock:{agent_region}::foundation-model/{foundational_model}"
                    ]
                }
            ]
        }
        role_name = f'bedrock_agent_{uuid.uuid4()}'
        iam_client = boto3.client('iam')
        response = iam_client.create_role(
            RoleName=role_name,
            AssumeRolePolicyDocument=assume_role_policy_document,
            Description='Role for Bedrock Agent'
        )
        iam_client.put_role_policy(
            RoleName=role_name,
            PolicyName=f'AmazonBedrockAgentBedrockFoundationModelPolicy_{uuid.uuid4()}',
            PolicyDocument=json.dumps(managed_policy)
        )
        time.sleep(2)
        return response.get('Role', {}).get('Arn', '')

    except Exception as exception:
        raise exception

agent_resource_role_arn = _create_agent_role(
            agent_region='us-west-2',
            foundational_model=foundational_model)

agent_resource_role_arn

'arn:aws:iam::151065682055:role/bedrock_agent_45592175-ece8-4e3f-9549-3da9907ab73f'

### Step 4: Now create the Bedrock Agent and initialize the AgentExecutor

In [24]:
from langchain.agents import AgentExecutor
from langchain_aws.agents import BedrockAgentsRunnable

#setting enable trace to True, if you do not want all the trace events then set to False.
my_enable_trace = True

agent = BedrockAgentsRunnable.create_agent(
            agent_name="mortgage_interest_rate_agent",
            agent_resource_role_arn=agent_resource_role_arn,
            foundation_model=foundational_model,
            instruction="""
            You are an agent who helps with getting the mortgage rate based on the current asset valuation""",
            tools=tools,
            enable_trace=my_enable_trace
        )

agent_executor = AgentExecutor(agent=agent, tools=tools, return_intermediate_steps=True) 
agent_executor


AgentExecutor(verbose=False, agent=RunnableAgent(runnable=BedrockAgentsRunnable(agent_id='QBXLCYJY2T', client=<botocore.client.AgentsforBedrockRuntime object at 0x11cbc3710>, enable_trace=True), input_keys_arg=[], return_keys_arg=[], stream_runnable=True), tools=[StructuredTool(name='AssetDetail::getAssetValue', description='Get the asset value for an owner id\n\nArgs:\n    asset_holder_id: The asset holder id\n\nReturns:\n    The asset value for the given asset holder', args_schema=<class 'langchain_core.utils.pydantic.AssetDetail::getAssetValue'>, func=<function get_asset_value at 0x11ef4cc20>), StructuredTool(name='AssetDetail::getMortgageRate', description='Get the mortgage rate based on asset value\n\nArgs:\n    asset_holder_id: The asset holder id\n    asset_value: The value of the asset\n\nReturns:\n    The interest rate for the asset holder and asset value', args_schema=<class 'langchain_core.utils.pydantic.AssetDetail::getMortgageRate'>, func=<function get_mortgage_rate at 0x1

### Step 5: Invoke the agent

In [25]:
output = agent_executor.invoke({"input": "what is my mortgage rate for id AVC-1234"})
output

{'input': 'what is my mortgage rate for id AVC-1234',
 'output': 'The mortgage rate for asset holder id AVC-1234 with an asset value of 100K is 8.87%.',
 'intermediate_steps': [(BedrockAgentAction(tool='AssetDetail::getAssetValue', tool_input={'asset_holder_id': 'AVC-1234'}, log='{"returnControl": {"invocationId": "95dc9a72-8a90-41f2-8317-a7e2e98fa5d7", "invocationInputs": [{"functionInvocationInput": {"actionGroup": "AssetDetail", "actionInvocationType": "RESULT", "function": "getAssetValue", "parameters": [{"name": "asset_holder_id", "type": "string", "value": "AVC-1234"}]}}]}}', session_id='9ac1c763-d1af-42d0-ba6c-f544d19613f1', trace_log='[{"agentAliasId": "TSTALIASID", "agentId": "QBXLCYJY2T", "agentVersion": "DRAFT", "sessionId": "9ac1c763-d1af-42d0-ba6c-f544d19613f1", "trace": {"orchestrationTrace": {"modelInvocationInput": {"inferenceConfiguration": {"maximumLength": 2048, "stopSequences": ["</function_calls>", "</answer>", "</error>"], "temperature": 0.0, "topK": 250, "topP": 

#### Commentary
Since we specified `return_intermediate_steps=True`, we get the intermediate steps from the agents. The first time agent is invoked, it return a `BedrockAgentAction` to determine the asset value for the given asset holder. After the asset value is determined, the agent again return the control for the next tool with asset value and asset holder id to determine the interest rate. Once the interest rate is returned, the agent returns the final output `The mortgage rate for the asset holder ID AVC-1234 with an asset value of 100K is 8.87%.`

### Step 6: Cleanup

def delete_agent_role(agent_resource_role_arn: str):
    """
    Delete agent resource role

    Args:
       agent_resource_role_arn: Associated Agent execution role arn
    """
    try:
        iam_client = boto3.client('iam')
        role_name = agent_resource_role_arn.split('/')[-1]
        inline_policies = iam_client.list_role_policies(RoleName=role_name)
        for inline_policy_name in inline_policies.get('PolicyNames', []):
            iam_client.delete_role_policy(
                RoleName=role_name,
                PolicyName=inline_policy_name
            )
        iam_client.delete_role(
            RoleName=role_name
        )
    except Exception as exception:
        raise exception


def delete_agent(agent_id):
    bedrock_client = boto3.client('bedrock-agent')
    bedrock_client.delete_agent(agentId=agent_id, skipResourceInUseCheck=True)


delete_agent(agent_id=agent.agent_id)
delete_agent_role(agent_resource_role_arn=agent_resource_role_arn)